### Install and import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
try:
  !pip install git+https://github.com/optuna/optuna.git
  !pip install scikit-optimize
except:
  print('')

%tensorflow_version 1.x
import tensorflow as tf
import optuna
import numpy as np
import os
import random
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
import sqlite3
from sqlite3 import Error
import gc
import json


np.random.seed(2020)
random_state = 52
PATH_COLAB = "/content/drive/My Drive/Deep Learning Project/progetto/"
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patien
print("Tensorflow version " + tf.__version__)
PATH_OPTUNA = PATH_COLAB+"MobileNet_optuna/GP_LCB/opt_history_GP_LCB_batch32.db"

### Definition of functions

In [ ]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

def preprocessing(X, Y, img_rows, img_cols, color_type=3):
    
    Y = np.array(Y, dtype=np.uint8)

    if color_type == 1:
        X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 1)
    else:
        X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 3)


    Y = tf.keras.utils.to_categorical(Y, 10)
    X = X.astype('float32')
    X /= 255
    
    return X, Y

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('Version sqlite: {}'.format(sqlite3.version))
        print('Connected')
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

# Reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    global data_generator, X_val, Y_val, Y_train, model, datagen

    try:
        del data_generator, datagen, X_val, Y_val, Y_train, model
    except:
        pass
    
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

class KerasPruningCallback_modified(optuna.integration.KerasPruningCallback):

      def on_epoch_end(self, epoch, logs=None):
            
        global is_prune
        logs = logs or {}
        current_score = logs.get(self._monitor)
        if current_score is None:
            return
        self._trial.report(float(current_score), step=epoch)
        if self._trial.should_prune():
            is_prune = True
            return

In [0]:
def create_model(dense_1, dropout_rate_1, dense_2, dropout_rate_2, activation_selected, learning_rate, momentum):
  
    base_model = tf.keras.applications.MobileNet(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

    if activation_selected == "relu":
    model = tf.keras.Sequential([
      base_model,
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_1),
      tf.keras.layers.ReLU(),
      tf.keras.layers.Dropout(dropout_rate_1),
      tf.keras.layers.Dense(dense_2),
      tf.keras.layers.ReLU(),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(dropout_rate_2),
      tf.keras.layers.Dense(10, activation = 'softmax')
    ])
    else:
    model = tf.keras.Sequential([
      base_model,
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_1),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.Dropout(dropout_rate_1),
      tf.keras.layers.Dense(dense_2),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(dropout_rate_2),
      tf.keras.layers.Dense(10, activation = 'softmax')
    ])


    model.compile(loss = "categorical_crossentropy",
            optimizer = tf.keras.optimizers.SGD(lr=learning_rate,momentum=momentum),
            metrics = ['accuracy'])

    return model

def objective(trial):

    reset_keras()
    trial_num = trial.number

    print("Trial: ",trial_num)

    batch_size = 32
    epochs = 20
    is_nan = False
    sum_score_log_loss = 0
    global is_prune
    is_prune = False

    # parameters to optimize
    dropout_rate_1 = trial.suggest_discrete_uniform("dropout_rate_1", 0, .5, .1)
    dropout_rate_2 = trial.suggest_discrete_uniform("dropout_rate_2", 0, .5, .1)

    activation_options = ["relu", "leakyRelu"]
    activation_selected = trial.suggest_categorical("activation", activation_options)

    dense_1 = trial.suggest_int("n_units_1", 64, 512)
    dense_2 = trial.suggest_int("n_units_2", 32, 256)

    learning_rate=trial.suggest_loguniform("lr",1e-4, 1e-2)
    momentum=trial.suggest_loguniform("momentum",0.5,0.9)

    for k in range(5,0,-1):
      
        print("Fold: ", k)
        global data_generator, X_train, X_val, Y_val, Y_train, model, datagen
        X_train, Y_train, X_val, Y_val = restore_data(PATH_COLAB + "data/k_" + str(k) + ".dat")
        X_train, Y_train = preprocessing(X_train, Y_train, 128, 128, 3)
        X_val, Y_val = preprocessing(X_val, Y_val, 128, 128, 3)

        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            height_shift_range=0.5,
            width_shift_range = 0.5,
            zoom_range = 0.5,
            rotation_range=20
        )
      
        data_generator = datagen.flow(X_train, Y_train, batch_size = batch_size)

        length_train = len(X_train)
        del X_train

        model = create_model(dense_1, dropout_rate_1, dense_2, dropout_rate_2, activation_selected, learning_rate, momentum)

        class_weights = class_weight.compute_sample_weight(class_weight='balanced', y=Y_train)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(PATH_COLAB + "MobileNet_optuna/trials/opt_best_t" +str(trial_num) + "_k" + str(k) + ".h5", monitor='val_loss', verbose=0, save_best_only=True, mode='min')
        callbacks = [KerasPruningCallback_modified(trial, 'val_loss'), early_stopping, checkpoint] 

        model.fit_generator(data_generator, steps_per_epoch = length_train // batch_size, 
                epochs=epochs, shuffle=True, verbose=0, validation_data=(X_val, Y_val),
                class_weight=class_weights, callbacks=callbacks)

        predictions_val = model.predict(X_val, batch_size=batch_size, verbose=0)

        if np.isnan(predictions_val).any():
            is_nan = True
            reset_keras()
            break

        score_log_loss = log_loss(Y_val, predictions_val)      
        print('Score Log Loss: ', score_log_loss)
        sum_score_log_loss += score_log_loss*X_val.shape[0]
        reset_keras()

        if is_prune:
            break

    if is_nan:
        return float('nan')

    elif is_prune:
        raise optuna.exceptions.TrialPruned("Pruned.")

    else:
        score_log_loss = sum_score_log_loss/22424
        print("Weighted avg log loss: ", score_log_loss)
        return score_log_loss


In [11]:
create_connection(PATH_OPTUNA)

Version sqlite: 2.6.0
Connected


### Creation of study for optimization

In [0]:
study_name="study_optuna_GP_LCB_batch32"

study = optuna.create_study(study_name=study_name,direction='minimize',
                            sampler = optuna.integration.SkoptSampler(independent_sampler=optuna.samplers.TPESampler(),
                                                                      warn_independent_sampling = True,
                                                                      skopt_kwargs={'base_estimator':'GP', 'acq_func':'LCB'}),
                            storage='sqlite:///'+PATH_OPTUNA,
                            load_if_exists=True)


In [7]:
study.optimize(objective, n_trials=54)

Delete object:  195
Trial:  44
Fold:  5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
17227776/17225924 [==============================] - 1s 0us/step
Score Log Loss:  0.7017031816901135
Delete object:  31
Fold:  4
Score Log Loss:  0.5968580996671071
Delete object:  103836
Fold:  3
Score Log Loss:  0.3485784587148523
Delete object:  103836
Fold:  2
Score Log Loss:  0.3582712447837136
Delete object:  103836
Fold:  1
Score Log Loss:  0.9162574296086254
Delete object:  103836
Weighted avg log loss:  0.5834129726412479


[I 2020-04-24 08:59:02,390] Finished trial#44 with value: 0.5834129726412479 with parameters: {'activation': 'leakyRelu', 'dropout_rate_1': 0.0, 'dropout_rate_2': 0.0, 'lr': 0.009999999999999998, 'momentum': 0.5, 'n_units_1': 402, 'n_units_2': 256}. Best is trial#18 with value: 0.3857265598895732.


Delete object:  237
Trial:  45
Fold:  5
Score Log Loss:  0.49040473811834384
Delete object:  103828
Fold:  4
Score Log Loss:  0.44291076779804606
Delete object:  104225
Fold:  3
Score Log Loss:  0.33487992725675014
Delete object:  104225
Fold:  2
Score Log Loss:  0.25701524345048704
Delete object:  104225
Fold:  1
Score Log Loss:  0.5295130578379642
Delete object:  104225
Weighted avg log loss:  0.4070137003476277


[I 2020-04-24 10:13:19,061] Finished trial#45 with value: 0.4070137003476277 with parameters: {'activation': 'relu', 'dropout_rate_1': 0.0, 'dropout_rate_2': 0.1, 'lr': 0.009999999999999998, 'momentum': 0.8074490520982605, 'n_units_1': 453, 'n_units_2': 32}. Best is trial#18 with value: 0.3857265598895732.


Delete object:  237
Trial:  46
Fold:  5
Score Log Loss:  1.044307162545498


...

Delete object:  31
Delete object:  237
Trial:  52
Fold:  5
Score Log Loss:  1.0587384990534339


[I 2020-04-24 13:19:45,748] Setting status of trial#52 as TrialState.PRUNED. Pruned.


Delete object:  38
Delete object:  237
Trial:  53
Fold:  5
Score Log Loss:  1.0257681278411719


[I 2020-04-24 13:46:13,503] Setting status of trial#53 as TrialState.PRUNED. Pruned.


Delete object:  31


In [12]:
print("Number of finished trials: ", len(study.trials))

print("\nBest trial:")
trial = study.best_trial
print("\nValue: ", trial.value)
print("\nParams: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  54

Best trial:

Value:  0.3857265598895732

Params: 
    activation: relu
    dropout_rate_1: 0.1
    dropout_rate_2: 0.2
    lr: 0.00813603753142859
    momentum: 0.5188983231483192
    n_units_1: 387
    n_units_2: 107


In [19]:
study = optuna.load_study("study_optuna_GP_LCB_batch32", storage='sqlite:///'+PATH_OPTUNA)
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_activation,params_dropout_rate_1,params_dropout_rate_2,params_lr,params_momentum,params_n_units_1,params_n_units_2,state
0,0,0.757970,2020-04-22 20:44:37.753430,2020-04-22 23:03:03.685180,02:18:25.931750,relu,0.1,0.0,0.000149,0.694414,70,148,COMPLETE
1,1,0.607083,2020-04-22 23:03:03.742696,2020-04-23 00:47:35.458973,01:44:31.716277,leakyRelu,0.3,0.2,0.000614,0.623129,125,147,COMPLETE
2,2,0.510934,2020-04-23 00:47:35.516788,2020-04-23 02:35:46.911287,01:48:11.394499,relu,0.3,0.3,0.001292,0.646910,309,108,COMPLETE
3,3,0.536154,2020-04-23 02:35:46.980012,2020-04-23 03:45:54.659051,01:10:07.679039,relu,0.0,0.2,0.005280,0.590060,229,82,COMPLETE
4,4,NaN,2020-04-23 03:45:54.717133,NaT,NaT,relu,0.1,0.2,0.000159,0.525534,317,48,RUNNING
5,5,0.528575,2020-04-23 06:55:13.382383,2020-04-23 08:09:18.619707,01:14:05.237324,relu,0.2,0.1,0.008325,0.548890,302,256,COMPLETE
6,6,0.960867,2020-04-23 08:09:18.680864,2020-04-23 08:34:42.349132,00:25:23.668268,leakyRelu,0.4,0.3,0.000102,0.743691,120,165,PRUNED
7,7,0.709393,2020-04-23 08:34:42.563040,2020-04-23 09:05:06.390455,00:30:23.827415,leakyRelu,0.2,0.1,0.007145,0.504771,98,185,PRUNED
8,8,NaN,2020-04-23 08:53:11.506177,NaT,NaT,relu,0.2,0.1,0.008325,0.548890,302,256,RUNNING
9,9,0.991860,2020-04-23 09:05:06.594755,2020-04-23 09:25:34.388566,00:20:27.793811,leakyRelu,0.0,0.2,0.003136,0.671606,176,117,PRUNED


In [20]:
df.state.value_counts()

PRUNED      34
COMPLETE    16
RUNNING      4
Name: state, dtype: int64

In [ ]:
with open(PATH_COLAB+"best_params_mobilenet.json", 'w') as filename:
    json.dump(trial.params, filename)

### Visualization

In [13]:
import plotly

ly = plotly.graph_objs.Layout(height=700, width=1300)
fig = plotly.graph_objs.Figure(layout=ly)

optuna.visualization.plot_parallel_coordinate(study)

In [14]:
optuna.visualization.plot_contour(study, ['lr','momentum'])

In [15]:
optuna.visualization.plot_contour(study, ['activation', 'dropout_rate_1', 'dropout_rate_2', 'n_units_1','n_units_2','lr','momentum'])

In [16]:
optuna.visualization.plot_optimization_history(study)

In [17]:
optuna.visualization.plot_intermediate_values(study)

In [18]:
optuna.visualization.plot_slice(study)